In [35]:
import os
import re
import requests
import numpy as np
import pandas as pd
import datetime as dt
import plotly.express as px
from time import sleep
from functools import wraps, partial
from concurrent.futures import ProcessPoolExecutor

In [36]:
BASE_URL = f"https://api.nasdaq.com/api"

# Utils

In [37]:
def validator(function):
        @wraps(function)
        def wrapper(*args, **kwargs):
            try:
                return function(*args, **kwargs)
            except (AttributeError, TypeError) as e:
                pass
        return wrapper


In [38]:
def get_response(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.2 Safari/605.1.15" 
    }

    session = requests.Session()
    response = session.get(url=url, headers=headers)
    return response.json()


def get_dataframe(data):
    cols, rows = data["headers"], data["rows"]

    df = pd.DataFrame(data=rows, columns=list(cols.keys()))
    df.columns = list(cols.values())
    df.set_index(df.columns[0], inplace=True)
    return df


def snakecase_to_camelcase(s):
    return re.sub(r'_([a-z])', lambda x: x.group(1).upper(), s)


def string_to_float(s):
    return float(re.sub(r"[\$\,]+", "", s))


# Screener

In [39]:
@validator
def screener(limit=0, **kwargs):
    """
    Inputs:
        limit: 0 = all data, 1+ = number of rows (default=0)
        exchange: nasdaq, nyse, amex (default=None)
        marketcap: mega, large, mid, small, micro, nano (default=None)
        recommendation: strong_buy, hold, buy, sell, strong_sell (default=None)
        sector: technology, telecommunications, health_care, finance, real_estate,
                consumer_discretionary, consumer_staples, industrials,
                basic_materials, energy, utilities (default=None)
        region: africa, asia, australia_and_south_pacific, caribbean, europe,
                middle_east, north_america, south_america (default=None)
        country: argentina, armenia, australia, austria, belgium, bermuda, brazil,
                 canada, cayman_islands, chile, colombia, costa_rica, curacao,
                 cyprus, denmark, finland, france, germany, greece, guernsey,
                 hong_kong, india, indonesia, ireland, isle_of_man, israel, italy,
                 japan, jersey, luxembourg, macau, mexico, monaco, netherlands,
                 norway, panama, peru, philippines, puerto_rico, russia, singapore,
                 south_africa, south_korea, spain, sweden, switzerland, taiwan,
                 turkey, united_kingdom, united_states, usa (default=None)

    Example: screener(exchange="nasdaq", marketcap=["mega", "large", "mid"], sector=["technology", "health_care", "finance"])
    """
    parameters = {
        "exchange": ["nasdaq", "nyse", "amex"],
        "marketcap": ["mega", "large", "mid", "small", "micro", "nano"],
        "recommendation": ["strong_buy", "hold", "buy", "sell", "strong_sell"],
        "sector": ["technology", "telecommunications", "health_care", "finance", "real_estate", "consumer_discretionary", "consumer_staples", "industrials", "basic_materials", "energy", "utilities"],
        "region": ["africa", "asia", "australia_and_south_pacific", "caribbean", "europe", "middle_east", "north_america", "south_america"],
        "country": ["argentina", "armenia", "australia", "austria", "belgium", "bermuda", "brazil", "canada", "cayman_islands", "chile", "colombia", "costa_rica", "curacao",
                    "cyprus", "denmark", "finland", "france", "germany", "greece", "guernsey", "hong_kong", "india", "indonesia", "ireland", "isle_of_man", "israel", "italy",
                    "japan", "jersey","luxembourg", "macau", "mexico", "monaco", "netherlands", "norway", "panama", "peru", "philippines", "puerto_rico", "russia", "singapore",
                    "south_africa", "south_korea","spain", "sweden", "switzerland", "taiwan", "turkey", "united_kingdom", "united_states", "usa"]
    }

    url = f"{BASE_URL}/screener/stocks?limit={limit}"

    for i, v in kwargs.items():
        if isinstance(v, list):
            v = "|".join(v) if all(item in parameters[i] for item in v) else ""
        url += f"&{i}={v}"

    response = get_response(url)
    data = response["data"]["table"]

    df = get_dataframe(data)

    if isinstance(df, pd.DataFrame):
        df.index = [i.replace("/", ".") for i in df.index]

    return df


In [40]:
screener(exchange="nasdaq", marketcap=["mega", "large", "mid"], sector=["technology", "health_care", "finance"], country="usa")

# SingleBase

In [41]:
class SingleBase:
    def __init__(self, ticker):
        self._years = 10
        self._screener = screener()
        self._ticker = ticker.upper()

        self._base_url = BASE_URL
        self._quote_url = f"{self._base_url}/quote/{self._ticker}"
        self._company_url = f"{self._base_url}/company/{self._ticker}"
        self._analyst_url = f"{self._base_url}/analyst/{self._ticker}"

        self._total_days = 365 * self._years + 1
        self._trading_days = 252 * self._years
        self._start_date = (dt.date.today() - dt.timedelta(days=self._total_days)).strftime("%Y-%m-%d")

        if not self._ticker in self._screener.index:
            raise Exception(f"There is no company with a symbol {self._ticker}.")


    # Stock Info
    @validator
    def _get_info(self):
        return self._screener.filter(items=[self._ticker], axis=0)


    # Company Profile
    @validator
    def _get_profile(self):
        url = f"{self._company_url}/company-profile"

        response = get_response(url)
        data = list(response["data"].values())

        lbls = [i["label"] for i in data]
        vals = [j["value"] for j in data]

        df = pd.DataFrame(data=vals, index=lbls, columns=[self._ticker])
        return df


    # Key Data
    @validator
    def _get_summary(self):
        url = f"{self._quote_url}/summary?assetclass=stocks"

        response = get_response(url)
        data = list(response["data"]["summaryData"].values())

        lbls = [i["label"] for i in data]
        vals = [j["value"] for j in data]

        df = pd.DataFrame(data=vals, index=lbls, columns=[self._ticker])
        return df


    # Dividend History
    @validator
    def _get_dividends(self):
        url = f"{self._quote_url}/dividends?assetclass=stocks"

        response = get_response(url)
        data = response["data"]["dividends"]

        df = get_dataframe(data)
        return df


    # Historical Quotes (10-Years Maximum)
    @validator
    def _get_quotes_history(self):
        url = f"{self._quote_url}/historical?assetclass=stocks&fromdate={self._start_date}&limit={self._trading_days}"

        response = get_response(url)
        data = response["data"]["tradesTable"]

        df = get_dataframe(data)
        return df


    # Chart
    @validator
    def _get_chart(self, data):
        data = data.applymap(lambda x: string_to_float(x))
        data.index = pd.to_datetime(data.index)
        data = data.sort_index()
        chart = px.area(x=data.index, y=data["Close/Last"])

        axes_layout = {
                        "gridcolor": "rgb(240, 240, 240)",
                        "showspikes": True,
                        "spikecolor": "rgb(120, 120, 240)",
                        "spikemode": "across",
                        "spikesnap": "cursor",
                        "spikedash": "dash",
                        "spikethickness": 0.5
                        }

        chart.update_xaxes(
            axes_layout,
            title_text="Date",
            rangeselector={
                "buttons": [
                    {"count": 1, "label": "1M", "step": "month", "stepmode": "backward"},
                    {"count": 3, "label": "3M", "step": "month", "stepmode": "backward"},
                    {"count": 6, "label": "6M", "step": "month", "stepmode": "backward"},
                    {"count": 1, "label": "YTD", "step": "year", "stepmode": "todate"},
                    {"count": 1, "label": "1Y", "step": "year", "stepmode": "backward"},
                    {"count": 3, "label": "3Y", "step": "year", "stepmode": "backward"},
                    {"count": 5, "label": "5Y", "step": "year", "stepmode": "backward"},
                    {"label": "MAX", "step": "all"}]})

        chart.update_yaxes(axes_layout, title_text="Price", tickprefix="$")

        chart.update_layout(
            plot_bgcolor="rgb(250, 250, 250)",
            hovermode="x",
            spikedistance=-1,
            hoverdistance=-1,
            showlegend=False,
            title={
                "text": f"{self._ticker} Stock Price History",
                "y": 0.97,
                "x": 0.5,
                "xanchor": "center",
                "yanchor": "top"})

        chart.update_traces(hovertemplate="Date: %{x}<br>Price: %{y}")
        chart.show();


    # Financials (Annual/Quarterly)
    @validator
    def _get_financials(self, table=None, frequency=None):
        frequency = 2 if frequency == "quarterly" else 1

        url = f"{self._company_url}/financials?frequency={frequency}"

        response = get_response(url)

        if table == "balance_sheet":
            data = response["data"]["balanceSheetTable"]
        elif table == "cash_flow":
            data = response["data"]["cashFlowTable"]
        elif table == "financial_ratios":
            data = response["data"]["financialRatiosTable"]
        else:
            data = response["data"]["incomeStatementTable"]

        df = get_dataframe(data)
        return df


    # Earnings Per Share (Estimated vs Reported)
    @validator
    def _get_eps(self):
        url = f"{self._quote_url}/eps"

        response = get_response(url)
        data = response["data"]["earningsPerShare"]

        lbls = [i["period"] for i in data]
        vals = [(j["consensus"], j["earnings"]) for j in data]

        df = pd.DataFrame(data=vals, index=lbls, columns=["Estimated EPS", "Reported EPS"])
        return df


    # Earnings Forecast (Yearly/Quarterly)
    @validator
    def _get_earnings_forecast(self, frequency=None):
        url = f"{self._analyst_url}/earnings-forecast"

        response = get_response(url)
        data = response["data"]["quarterlyForecast"] if frequency == "quarterly" else response["data"]["yearlyForecast"]

        df = get_dataframe(data)
        return df


    # Price/Earnings Ratio
    @validator
    def _get_pe_ratio(self):
        url = f"{self._analyst_url}/peg-ratio"

        response = get_response(url)
        data = response["data"]["per"]

        lbls = [i["x"] for i in data["peRatioChart"]]
        vals = [j["y"] for j in data["peRatioChart"]]

        df = pd.DataFrame(data=vals, index=lbls, columns=[data["label"]])
        return df


    # Forecast P/E Growth Rates
    @validator
    def _get_pe_growth(self):
        url = f"{self._analyst_url}/peg-ratio"

        response = get_response(url)
        data = response["data"]["gr"]

        lbls = [f"{i['z']} {i['x']}" for i in data["peGrowthChart"]]
        vals = [j["y"] for j in data["peGrowthChart"]]

        df = pd.DataFrame(data=vals, index=lbls, columns=[data["title"]])
        return df


    # PEG Ratio
    @validator
    def _get_peg_ratio(self):
        url = f"{self._analyst_url}/peg-ratio"

        response = get_response(url)
        data = response["data"]["pegr"]

        lbls = data["label"]
        vals = data["pegValue"]

        df = pd.DataFrame(data=vals, index=[lbls], columns=["PEG Ratio"])
        return df


    # Short Interest
    @validator
    def _get_short_interest(self):
        url = f"{self._quote_url}/short-interest?assetClass=stocks"

        response = get_response(url)
        data = response["data"]["shortInterestTable"]

        df = get_dataframe(data)
        return df


    # Institutional Holdings
    @validator
    def _get_institutional_holdings(self, limit=5000, **kwargs):
        parameters = {
            "type_": ["total", "new", "increased", "decreased", "activity", "soldout"],
            "sortColumn": ["ownerName", "date", "sharesHeld", "sharesChange", "sharesChangePCT", "marketValue"],
            "sortOrder": ["asc", "desc"]
        }

        url = f"{self._company_url}/institutional-holdings?limit={limit}&tableonly=true"

        for i, v in kwargs.items():
            i = snakecase_to_camelcase(i.strip('_'))
            url += f"&{i}={v}"

        response = get_response(url)
        data = response["data"]["holdingsTransactions"]["table"]

        df = get_dataframe(data)
        return df


    # Insider Activity
    @validator
    def _get_insider_activity(self, limit=5000, **kwargs):
        parameters = {
            "type_": ["all", "buys", "sells"],
            "sortColumn": ["insider", "relation", "lastDate", "transactionType", "ownType", "sharesTraded", "lastPrice", "sharesHeld"],
            "sortOrder": ["asc", "desc"]
        }

        url = f"{self._company_url}/insider-trades?limit={limit}"

        for i, v in kwargs.items():
            i = snakecase_to_camelcase(i.strip('_'))
            url += f"&{i}={v}"

        response = get_response(url)
        data = response["data"]["transactionTable"]["table"]

        df = get_dataframe(data)
        return df


    # Revenue EPS
    @validator
    def _get_revenue_eps(self):
        limit = 1
        df = pd.DataFrame()

        while True:
            url = f"{self._company_url}/revenue?limit={limit}"

            response = get_response(url)
            data = response["data"]["revenueTable"]

            if not data:
                break

            tmp_df = get_dataframe(data)
            df = pd.concat([df, tmp_df], axis=1)
            limit += 1

        return df

    # Fair Value
    @validator
    def _get_fair_value(self, discount_rate):
        eps = self._get_earnings_forecast()["Consensus EPS* Forecast"]
        eps.index = pd.to_datetime(eps.index).to_period('M').to_timestamp('M')

        years_to_forecast = [dt.date.today().year + i for i in range(3)]

        if len(eps) > 3:
            eps = eps[eps.index.year.isin(years_to_forecast)]
        elif len(eps) < 3:
            eps_gr = eps.pct_change().mean()
            eps_last = eps.iloc[-1] * (1 + eps_gr)
            index_last = eps.index[-1] + dt.timedelta(days=365)
            eps = eps.append(pd.Series(eps_last, index=[index_last]))

        discount_periods = [round((eps.index[i].date() - dt.date.today()).days / 365, 2) for i in range(len(eps))]
        discount_periods.append(discount_periods[-1])

        pe_ratio = self._get_pe_ratio()
        terminal_pe = pe_ratio.iloc[-1, 0]

        terminal_value = terminal_pe * eps.values[-1]

        eps_arr = np.append(eps.values, terminal_value)
        rates_arr = pow(np.array([(1.0 + discount_rate)] * 4), discount_periods)

        pv_arr = eps_arr / rates_arr

        fair_value = round(sum(pv_arr), 2)

        last_sale = string_to_float(self._get_info()["Last Sale"][0])

        profile = self._get_profile()
        symbol = profile.loc["Symbol"][0]
        company_name = profile.loc["Company Name"][0]
        sector = profile.loc["Sector"][0]
        industry = profile.loc["Industry"][0]

        discount_rate = f"{discount_rate * 100}%"

        data = {
            "Company Name": company_name,
            "Sector": sector,
            "Industry": industry,
            "Discount Rate": discount_rate,
            "Current Price": last_sale,
            "Intrinsic Value": fair_value
        }

        df = pd.DataFrame(data.values(), index=data.keys(), columns=[self._ticker])
        return df


# Ticker

In [42]:
class Ticker(SingleBase):
    def __repr__(self):
        return "nasdaq.Ticker object <%s>" % self._ticker

    @property
    def info(self):
        return self._get_info()

    @property
    def profile(self):
        return self._get_profile()

    @property
    def summary(self):
        return self._get_summary()

    @property
    def dividends(self):
        return self._get_dividends()

    @property
    def quotes_history(self):
        return self._get_quotes_history()

    @property
    def chart(self):
        return self._get_chart(self._get_quotes_history())

    @property
    def income_statement_annual(self):
        return self._get_financials()

    @property
    def balance_sheet_annual(self):
        return self._get_financials(table="balance_sheet")

    @property
    def cash_flow_annual(self):
        return self._get_financials(table="cash_flow")

    @property
    def financial_ratios_annual(self):
        return self._get_financials(table="financial_ratios")

    @property
    def income_statement_quarterly(self):
        return self._get_financials(table="quarterly")

    @property
    def balance_sheet_quarterly(self):
        return self._get_financials(table="balance_sheet", frequency="quarterly")

    @property
    def cash_flow_quarterly(self):
        return self._get_financials(table="cash_flow", frequency="quarterly")

    @property
    def financial_ratios_quarterly(self):
        return self._get_financials(table="financial_ratios", frequency="quarterly")

    @property
    def eps(self):
        return self._get_eps()

    @property
    def earnings_forecast_yearly(self):
        return self._get_earnings_forecast()

    @property
    def earnings_forecast_quarterly(self):
        return self._get_earnings_forecast(frequency="quarterly")

    @property
    def pe_ratio(self):
        return self._get_pe_ratio()

    @property
    def pe_growth(self):
        return self._get_pe_growth()

    @property
    def peg_ratio(self):
        return self._get_peg_ratio()

    @property
    def short_interest(self):
        return self._get_short_interest()

    def institutional_holdings(self, **kwargs):
        """
        Inputs:
            limit: 1+ (default=5000)
            type_: total, new, increased, decreased, activity, soldout (default=total)
            sort_column: ownerName, date, sharesHeld, sharesChange, sharesChangePCT, marketValue (default=marketValue)
            sort_order: asc, desc (default=desc)
        
        Example: Ticker.institutional_holdings(limit=5000, type_="total", sort_column="marketValue", sort_order="desc")
        """
        return self._get_institutional_holdings(**kwargs)

    def insider_activity(self, **kwargs):
        """
        Inputs:
            limit: 1+ (default=5000)
            type_: all, buys, sells (default=all)
            sort_column: insider, relation, lastDate, transactionType, ownType, sharesTraded, lastPrice, sharesHeld (default=lastDate)
            sort_order: asc, desc (default=desc)
        
        Example: Ticker.insider_activity(limit=5000, type_="all", sort_column="lastDate", sort_order="desc")
        """
        return self._get_insider_activity(**kwargs)

    @property
    def revenue_eps(self):
        return self._get_revenue_eps()

    def fair_value(self, discount_rate=0.1):
        return self._get_fair_value(discount_rate)


In [43]:
company = Ticker("aapl")

In [44]:
company.fair_value()

,AAPL
Company Name,Apple Inc.
Sector,Technology
Industry,Computer Manufacturing
Discount Rate,10.0%
Current Price,141.86
Intrinsic Value,127.04


# S&P 500 Companies

In [45]:
def sp500_companies():
    url = "https://en.wikipedia.org/wiki/List_of_S&P_500_companies"
    data = pd.read_html(url)[0]
    return data

# MultiBase

In [46]:
class MultiBase:
    def __init__(self, tickers):
        tickers = tickers if isinstance(tickers, (list, set, tuple)) else tickers

        self._tickers = [ticker.upper() for ticker in tickers]
        self._workers = os.cpu_count() + 4


    @validator
    def _get_quotes_history(self, ticker):
        data = Ticker(ticker)._get_quotes_history()
        data = data.loc[:, "Close/Last"]
        data.rename(ticker, inplace=True)
        return data


    def _get_fair_value(self, discount_rate, ticker):
        sleep(0.2)
        data = Ticker(ticker)._get_fair_value(discount_rate)
        return data


    def _multiprocessing(self, function, discount_rate):
        with ProcessPoolExecutor(self._workers) as executor:
            df = pd.concat(list(executor.map(partial(function, discount_rate), self._tickers)), axis=1)
        return df


# Tickers

In [47]:
class Tickers(MultiBase):
    def __repr__(self):
        return "nasdaq.Tickers object <%s>" % ", ".join(self._tickers)

    @property
    def quotes_history(self):
        return self._multiprocessing(self._get_quotes_history)

    def fair_value(self, discount_rate=0.1):
        return self._multiprocessing(self._get_fair_value, discount_rate)


In [48]:
symbols = sp500_companies()["Symbol"].tolist()

In [49]:
companies = Tickers(symbols)

In [50]:
df = companies.fair_value(discount_rate=0.12)
df

,MMM,AOS,ABT,ABBV,ACN,ATVI,ADM,ADBE,ADP,AAP,...,WTW,GWW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Company Name,3M Company,A.O. Smith Corporation,Abbott Laboratories,AbbVie Inc.,Accenture plc,"Activision Blizzard, Inc",Archer-Daniels-Midland Company,Adobe Inc.,"Automatic Data Processing, Inc.",Advance Auto Parts Inc.,...,Willis Towers Watson Public Limited Company,"W.W. Grainger, Inc.","Wynn Resorts, Limited",Xcel Energy Inc.,Xylem Inc.,"Yum! Brands, Inc.",Zebra Technologies Corporation,"Zimmer Biomet Holdings, Inc.",Zions Bancorporation N.A.,Zoetis Inc.
Sector,Industrials,Industrials,Health Care,Health Care,Consumer Discretionary,Consumer Discretionary,None,Technology,Consumer Discretionary,Consumer Discretionary,...,Finance,Consumer Discretionary,Consumer Discretionary,Utilities,Industrials,Consumer Discretionary,Technology,Health Care,Finance,Health Care
Industry,Consumer Electronics/Appliances,Industrial Machinery/Components,Medical/Dental Instruments,Other Pharmaceuticals,Business Services,Recreational Games/Products/Toys,None,Computer Software: Prepackaged Software,Business Services,Auto & Home Supply Stores,...,Specialty Insurers,Telecommunications Equipment,Hotels/Resorts,Power Generation,Fluid Controls,Restaurants,Computer peripheral equipment,Industrial Specialties,Major Banks,Biotechnology: Pharmaceutical Preparations
Discount Rate,12.0%,12.0%,12.0%,12.0%,12.0%,12.0%,12.0%,12.0%,12.0%,12.0%,...,12.0%,12.0%,12.0%,12.0%,12.0%,12.0%,12.0%,12.0%,12.0%,12.0%
Current Price,112.93,60.52,111.0,147.79,273.16,74.64,85.57,358.17,228.01,149.11,...,253.57,559.85,98.59,68.56,102.54,129.2,310.57,124.9,52.02,165.51
Intrinsic Value,104.58,56.74,91.21,148.44,234.84,56.63,86.95,292.67,196.21,155.72,...,240.25,516.42,76.17,60.56,92.48,117.65,293.02,114.69,53.96,137.4


In [52]:
df.to_csv(f"stock-fair-values-drate12-{dt.date.today()}.csv", encoding="utf-8-sig")